In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import sys
sys.path.append('..')

from helpers import utils, pipelines, models

from sklearn.model_selection import train_test_split

import multiprocessing

from data_augmentaion.data_augmentator import DataAugmentor 

import json

import time

### Define research parameters

In [3]:
regression_task = False
# continuous_features = []
metric = 'accuracy'
test_size_proportion = 0.2
augment_sample = 0.5

search_pipelines = pipelines.get_classification_pipelines()
search_parameters = models.parameters

settings = [
    {'method': 'random'},
    {'method': 'smote'},
    {'method': 'cf_random'},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 1}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 0.2}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 1}},
]

In [4]:
df = pd.read_csv(r"..\datasets\wine.csv")
df.drop(columns="Id",inplace=True)
target = 'quality'

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[target] = le.fit_transform(df[target])

#### split data

In [6]:
# all data
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_proportion, random_state=42)

In [7]:
continuous_features = X.columns.tolist()

In [8]:
display(y_train.value_counts())

2    387
3    363
4    117
1     27
5     14
0      6
Name: quality, dtype: int64

In [9]:
full_results_df = pd.DataFrame()

### Whole dataset scores

In [10]:
print(f'whole adult dataset {metric} scores:')
_, whole_scores = utils.fit_and_evaluate(X_train, y_train, X_test, y_test,
                    search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
whole_dataset_result_df = pd.DataFrame.from_dict(whole_scores, orient='index', columns=[f'full data {metric} score'])
display(whole_dataset_result_df.T)

whole adult dataset accuracy scores:


,lg,rf,xgb
full data accuracy score,0.624454,0.624454,0.598253


In [11]:
full_results_df = pd.concat([full_results_df, whole_dataset_result_df], axis=1)

### running experiments

In [12]:
best_estimators = {}
best_scores = {}
for i, s in enumerate(settings):
    start = time.time()
    print(f'{i} / {len(settings)}, {s}', end=' ')
    augmentor = DataAugmentor(X_train, y_train, X_test, y_test,
                             method=s['method'], regression=regression_task,
                             continuous_feats=continuous_features,
                             cf_scoring = metric,
                             kw_args=s.get('kw_args', {})
                             )

    X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
    best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
    utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_test, y_test,
                            search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
    result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'],
                                                orient='index',
                                                columns=[f'{json.dumps((list(s.values())))} balanced {metric} score'])

    X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
    best_estimators[f'{i}'], best_scores[f'{i}'] = \
        utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_test, y_test,
                               search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
    result_df = pd.DataFrame.from_dict(best_scores[f'{i}'],
                                       orient='index',
                                       columns=[f'{json.dumps((list(s.values())))} {metric} score'])
    
    full_results_df = pd.concat([full_results_df, result_df_balanced, result_df], axis=1)
    print(f'{time.time() - start} seconds for settings {i}')


0 / 9, {'method': 'random'} 22.023993492126465 seconds for settings 0
1 / 9, {'method': 'smote'} 22.077296018600464 seconds for settings 1
2 / 9, {'method': 'cf_random'} 0/24


100%|██████████| 1/1 [00:00<00:00,  8.40it/s]


0/270


100%|██████████| 1/1 [00:00<00:00,  3.05it/s]


No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


...

100%|██████████| 1/1 [00:00<00:00,  9.24it/s]


350/360


100%|██████████| 1/1 [00:00<00:00, 11.48it/s]


0/373


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
full_results_df

In [ ]:
full_results_df.to_csv(rf'../log/experiment_multiclass_wine_{metric}_testsize{test_size_proportion}_augmentsample{augment_sample}.csv')